In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats.mstats import winsorize
from sklearn.preprocessing import MinMaxScaler

# `stocks` dataset

In [ ]:
stocks = pd.read_csv("stocks.csv")  # Load stocks dataset

In [ ]:
stocks["q"] = pd.to_datetime(stocks["q"])  # Update 'q' column so that it is a date

# `us_firms` dataset

In [ ]:
us_firms = pd.read_csv("usfirms2022.csv")  # Load US firms dataset

In [ ]:
# Rename columns to more familiar names
us_firms = us_firms.rename(columns = {"Ticker": "firm", "Sector NAICS\nlevel 1": "industry"})

# Merge `us_firms` and `stock` dataset

In [ ]:
stocks = stocks.merge(us_firms[["firm", "industry"]], on="firm")

In [ ]:
stocks = stocks[stocks["industry"] != "-"]

# Set multiindex

In [ ]:
multiindex = ["industry", "firm", "q"]

In [ ]:
stocks.set_index(multiindex, inplace=True)  # Set multidimensional indices

In [ ]:
stocks = stocks.sort_values(by=multiindex)  # Sort indices

In [ ]:
stocks.head()

# Get finance metrics

In [ ]:
stocks["cc returns"] = np.log(stocks["adjprice"]) - np.log(stocks.groupby(level=1)["adjprice"].shift(1))

In [ ]:
stocks["market value"] = stocks["originalprice"] * stocks["sharesoutstanding"]

In [ ]:
stocks["book value"] = stocks["totalassets"] - stocks["totalliabilities"]

In [ ]:
stocks["book-to-market ratio"] = stocks["book value"] / stocks["market value"]

In [ ]:
stocks["ebit"] = stocks["revenue"] - stocks["cogs"] - stocks["sgae"] - stocks["otheropexp"]

In [ ]:
stocks["sales annual growth"] = stocks["revenue"] \
    / stocks.groupby(level=1)["revenue"].shift(4).replace(0, np.nan) - 1

In [ ]:
stocks["operating profit growth"] = stocks["ebit"] \
    / stocks.groupby(level=1)["ebit"].shift(4).replace(0, np.nan) - 1

In [ ]:
stocks["operating profit margin"] = stocks["ebit"] / stocks["revenue"].replace(0, np.nan)  # Avoid division by 0

In [ ]:
stocks["short financial leverage"] = stocks["shortdebt"] / stocks["totalassets"].replace(0, np.nan)

In [ ]:
stocks["long financial leverage"] = stocks["longdebt"] / stocks["totalassets"].replace(0, np.nan)

In [ ]:
stocks["net income"] = stocks["ebit"] - stocks["finexp"] - stocks["incometax"] + stocks["extraincome"]

In [ ]:
stocks["eps"] = stocks["net income"] / stocks["sharesoutstanding"].replace(0, np.nan)

In [ ]:
stocks["epsp"] = stocks["eps"] / stocks["originalprice"].replace(0, np.nan)

In [ ]:
stocks.head()

# General questions

## Firm size

### Market value

In [ ]:
hist = sns.histplot(
    stocks, x="market value", bins=60,
    hue=stocks.index.get_level_values("industry"), common_norm=False
)

sns.move_legend(hist, "lower center", bbox_to_anchor=(0.5, 1.05), title="Industry")

plt.show()

In [ ]:
# Log scale net income
hist = sns.histplot(
    stocks, x="market value", bins=60,
    hue=stocks.index.get_level_values("industry"), common_norm=False, log_scale=True
)

sns.move_legend(hist, "lower center", bbox_to_anchor=(0.5, 1.05), title="Industry")

plt.show()

### Book value 

In [ ]:
hist = sns.histplot(
    stocks, x="book value", bins=60,
    hue=stocks.index.get_level_values("industry"), common_norm=False
)

sns.move_legend(hist, "lower center", bbox_to_anchor=(0.5, 1.05), title="Industry")

plt.show()

## Sales performance and profitability

### Net income

In [ ]:
hist = sns.histplot(
    stocks, x="net income", bins=60,
    hue=stocks.index.get_level_values("industry"), common_norm=False
)

sns.move_legend(hist, "lower center", bbox_to_anchor=(0.5, 1.05), title="Industry")

plt.show()

### Earnings per share

In [ ]:
hist = sns.histplot(
    stocks, x="eps", bins=60,
    hue=stocks.index.get_level_values("industry"), common_norm=False
)

sns.move_legend(hist, "lower center", bbox_to_anchor=(0.5, 1.05), title="Industry")

plt.show()

# Specific questions

## Descriptive statistics

### Firms by industry

In [ ]:
last_quarter = stocks.index.levels[2][-1]

In [ ]:
stocks_last_quarter = stocks.loc[:, :, last_quarter]

In [ ]:
stocks_last_quarter

In [ ]:
hist = sns.histplot(
    data=stocks_last_quarter,
    y=stocks_last_quarter.index.get_level_values("industry"),
    hue=stocks_last_quarter.index.get_level_values("industry"),
    common_norm=False
)

sns.move_legend(hist, "lower center", bbox_to_anchor=(0.5, 1.0), title="Industry")

plt.show()

### Book value by industry

In [ ]:
hist = sns.histplot(
    stocks_last_quarter, x="book value", bins=60,
    hue=stocks_last_quarter.index.get_level_values("industry"), common_norm=False
)

sns.move_legend(hist, "lower center", bbox_to_anchor=(0.5, 1.0), title="Industry")

plt.show()

### Market value by industry

In [ ]:
hist = sns.histplot(
    stocks_last_quarter, x="market value", bins=60,
    hue=stocks_last_quarter.index.get_level_values("industry"), common_norm=False
)

sns.move_legend(hist, "lower center", bbox_to_anchor=(0.5, 1.0), title="Industry")

plt.show()

By examining the above histograms, it can be seen that both the market value and book value of the firms across different industries are skewed to the left. Because of this, to get a sense of the typical of these metrics, the median of both metrics is measured.

In [ ]:
stocks_last_quarter.index.levels[0].values

In [ ]:
typical_market_value = {
    industry: stocks_last_quarter.loc[industry, "market value"].median()
    for industry in stocks_last_quarter.index.levels[0].values
}

In [ ]:
typical_market_value = pd.DataFrame(
    data={
        "industry": typical_market_value.keys(),
        "median market value": typical_market_value.values()
    }
)

In [ ]:
typical_book_value = {
    industry: stocks_last_quarter.loc[industry, "book value"].median()
    for industry in stocks_last_quarter.index.levels[0].values
}

In [ ]:
typical_book_value = pd.DataFrame(
    data={
        "industry": typical_book_value.keys(),
        "median book value": typical_book_value.values()
    }
)

In [ ]:
typical_book_value.head()

In [ ]:
bar = sns.barplot(data=typical_market_value, x="median market value", y="industry", hue="industry", dodge=False)

sns.move_legend(bar, "lower center", bbox_to_anchor=(0.5, 1.0), title="Industry")

plt.show()

In [ ]:
bar = sns.barplot(data=typical_book_value, x="median book value", y="industry", hue="industry", dodge=False)

sns.move_legend(bar, "lower center", bbox_to_anchor=(0.5, 1.0), title="Industry")

plt.show()

In [ ]:
typical_firm_size = typical_market_value.merge(typical_book_value, on="industry")

In [ ]:
typical_firm_size.head()

## Statistical modeling

### Winsorization

In [ ]:
stocks_manufacturing = stocks.loc["Manufacturing"]

In [ ]:
stocks_manufacturing.info()

In [ ]:
stocks_manufacturing = stocks_manufacturing.clip(  # Winsorize dataset to remove most outliers
    lower=stocks_manufacturing.quantile(0.01),
    upper=stocks_manufacturing.quantile(0.99),
    axis=1
)

In [ ]:
stocks_manufacturing["firm size"] = np.where(
    stocks_manufacturing["market value"] < stocks_manufacturing["market value"].quantile(1/3),
    "small",
    "medium"
)

In [ ]:
stocks_manufacturing["firm size"] = np.where(
    stocks_manufacturing["market value"] >= stocks_manufacturing["market value"].quantile(2/3),
    "big",
    stocks_manufacturing["firm size"]
)

In [ ]:
stocks_manufacturing["firm size"].value_counts()

In [ ]:
dummy_columns = pd.get_dummies(stocks_manufacturing["firm size"])

In [ ]:
stocks_manufacturing = stocks_manufacturing.merge(dummy_columns, on=["firm", "q"])

In [ ]:
dependent_variable = "cc returns"

In [ ]:
independent_variables = [
    "eps",
    "net income",
    "operating profit margin",
    "operating profit growth",
    "sales annual growth", 
    "ebit",
    "short financial leverage",
    "long financial leverage",
    "book-to-market ratio"
]

In [ ]:
dummies = [
    "small",
    "medium",
    "big"
]

In [ ]:
columns = [dependent_variable] + independent_variables + dummies

In [ ]:
stats_model = stocks_manufacturing[columns]

Apply min-max normalization to values.

In [ ]:
stats_model = pd.DataFrame(
    MinMaxScaler().fit_transform(stats_model),
    columns=columns
)

In [ ]:
stats_model.describe()

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(15,12))

n_fig = 0

for ivar in independent_variables:
    splot = sns.scatterplot(
        data=stats_model,
        x=ivar,
        y=dependent_variable,
        ax=axes[n_fig // 3, n_fig % 3]
    )
    
    n_fig += 1
    
fig.tight_layout()

As it can be seen from the above histogram, there is no linear regression in any of the considered parameters. However, to train the linear regression for this deliverable, `net income`, `short financial leverage`, `sales annual growth`, and `eps` will be used for the model as they appear to have at least some degree of linear behaviour compared to the rest of variables that do not seem to be lineary related.

In [ ]:
independent_variables_model = ["eps", "net income", "short financial leverage", "sales annual growth"]

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(15,12))

n_fig = 0

for ivar in independent_variables_model:
    hist = sns.histplot(stats_model, x=ivar, ax=axes[n_fig])
    n_fig += 1
    
plt.tight_layout()

From the above histograms, it is possible to get a sense of the typical values for each of the factors. Because three of the histograms are skewed to the left, the typical value of these factor will correspond to the median. For the `eps` factor the mean is more appropiate sinde the distribution appears to have a normal behaviour.

Another remarkable insight is that histogram still present some outliers at their tails even after applying winsorization of 1% to the data.

In [ ]:
typical_eps = stats_model["eps"].mean()

In [ ]:
typical_net_income = stats_model["net income"].median()

In [ ]:
typical_short_financial_leverage = stats_model["short financial leverage"].median()

In [ ]:
typical_sales_annual_growth = stats_model["sales annual growth"].median()

In [ ]:
print("Typical eps:", typical_eps)

In [ ]:
print("Typical net income:", typical_net_income)

In [ ]:
print("Typical short financial leverage:", typical_short_financial_leverage)

In [ ]:
print("Typical sales annual growth:", typical_sales_annual_growth)

Additionally, to get a sense of the dispersion of the chosen factor, the standard deviation can be calculated for each factor.

In [ ]:
eps_std = stats_model["eps"].std()

In [ ]:
net_income_std = stats_model["net income"].std()

In [ ]:
short_financial_leverage_std = stats_model["short financial leverage"].std()

In [ ]:
sales_annual_growth_std = stats_model["sales annual growth"].std()

In [ ]:
print("eps standard deviation:", eps_std)

In [ ]:
print("Net income standard deviation:", net_income_std)

In [ ]:
print("Short financial leverage standard deviation:", short_financial_leverage_std)

In [ ]:
print("Sales annual growth standard deviation:", sales_annual_growth_std)

# Multiple regression

In [80]:
stats_model

,cc returns,eps,net income,operating profit margin,operating profit growth,sales annual growth,ebit,short financial leverage,long financial leverage,book-to-market ratio
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.357001,0.443804,0.152225,0.997883,NaN,NaN,0.119699,0.160442,0.000000,0.226851
2,0.318377,0.439928,0.148402,0.997848,NaN,NaN,0.118574,0.154759,0.000000,0.258068
3,0.622110,0.490727,0.200534,0.998123,NaN,NaN,0.188298,0.226009,0.000000,0.254107
4,0.221041,0.439149,0.148054,0.997942,NaN,NaN,0.137973,0.138525,0.000000,0.324214
...,...,...,...,...,...,...,...,...,...,...
140836,0.631831,0.349075,0.091074,NaN,0.430979,NaN,0.056769,0.004818,0.005399,0.326930
140837,0.428203,0.346688,0.090855,NaN,0.454190,NaN,0.056664,0.005293,0.005172,0.345213
140838,0.331962,0.353333,0.091469,NaN,0.445735,NaN,0.057050,0.005909,0.004932,0.401404
140839,0.359187,0.355488,0.091582,NaN,0.455012,NaN,0.057031,0.006493,0.004500,0.459124


In [77]:
X = stats_model[independent_variables_model]

In [81]:
y = stats_model[[dependent_variable]]

---

# EDA

## US firms by indutry

---

# Glossary

## Firm size measures

$$
\text{book value} = \text{total assets} - \text{total liabilities}
$$

$$
\begin{aligned}
\text{market value} &= \text{historical stock price} \times |\text{shares}|\\
                    &= \text{original price} \times \text{share outstanding}
\end{aligned}
$$

## Profit margin measures

$$
\text{operating profit margin} = \frac{\text{operating profit}}{\text{sales}}
$$

$$
\text{operating profit} = \text{revenue} - \text{cogs} - \text{sgae}
$$

$$
\text{cogs} = \text{cost of good sold} = \text{variable cost}
$$

$$
\text{sgae} = \text{sales and general administrative expenses} = \text{fixe costs}
$$

$$
\begin{aligned}
\text{ebit} &= \text{earnings before interest and taxes} = \text{opearting profit} \\
            &= \text{revenue} - \text{cogs} - \text{sgae} - \text{otheropexp}
\end{aligned}
$$

$$
\text{operating profit margin} = \text{opm} = \frac{\text{ebit}}{\text{revenue}}
$$

$$
\text{profit margin} = \frac{\text{net income}}{\text{sales}}
$$